# Huấn luyện có giám sát với SFTTrainer

Bài học này se dạy bạn các huấn luyện mô hình `HuggingFaceTB/SmolLM2-135M` bằng `SFTTrainer` trong thư viện `trl`.  Các cell trong notebook này sẽ chạy và huấn luyện mô hình. Bạn có thể chọn độ khó bằng cách thử nghiệm với các bộ dữ liệu khác nhau.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập: Fine-Tuning SmolLM2 với SFTTrainer</h2>
    <p>Chọn một bộ dự liệu từ Hugging Face hub và huấn luyện một mô hình trên bộ dữ liệu đó. </p>
    <p><b>Các bài tập</b></p>
    <p>🐢 Sử dụng bộ dữ liệu `HuggingFaceTB/smoltalk`</p>
    <p>🐕 Thử nghiệm với bộ dữ liệu `bigcode/the-stack-smol` và huấn luyện một mô hình sinh code trên tập con cụ thể `data/python`.</p>
    <p>🦁 Chọn một bộ dữ liệu liên quan đến một lĩnh vực mà bạn quan tâm</p>
</div>

In [3]:
# Cài đặt các thư viện cần thiết
!pip install transformers datasets trl huggingface_hub

# Đăng nhập vào Hugging Face
from huggingface_hub import login

login()

# Để thuận tiện, bạn có thể tạo một biến môi trường chứa `token hub` của bạn dưới dạng HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [18]:
# Các thư viện cần thiết
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Tải mô hình và tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Thiết lập định dạng chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Đặt tên cho mô hình huấn luyện để lưu &/ tải lên
finetune_name = "SmolLM2-FT-o1"
finetune_tags = ["smol-course", "module_1"]

# Sinh văn bản với Mô hình gốc

Ở đây chúng ta sẽ thử nghiệm mô hình gốc chưa được huấn luyện trên định dạng chat.

In [19]:
# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "Increasing the radius of a cylinder by $6$ units increased the volume by $y$ cubic units. Increasing the height of the cylinder by $6$ units also increases the volume by $y$ cubic units. If the original height is $2$, then the original radius is: $\text{(A) } 2 \qquad \text{(B) } 4 \qquad \text{(C) } 6 \qquad \text{(D) } 6\pi \qquad \text{(E) } 8$"

# Định dạng
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tạo phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=2048)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Increasing the radius of a cylinder by $6$ units increased the volume by $y$ cubic units. Increasing the height of the cylinder by $6$ units also increases the volume by $y$ cubic units. If the original height is $2$, then the original radius is: $	ext{(A) } 2 \qquad 	ext{(B) } 4 \qquad 	ext{(C) } 6 \qquad 	ext{(D) } 6\pi \qquad 	ext{(E) } 8$

Question 10:

A cylinder of radius $10$ cm and height $10$ cm is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a uniform material. The cylinder is to be made of a u

## Chuẩn bị dữ liệu

Chúng ta sẽ tải một bộ dữ liệu mẫu và định dạng nó cho việc huấn luyện. Bộ dữ liệu cần được cấu trúc với các cặp đầu vào - đầu ra, trong đó mỗi đầu vào là một chỉ thị và đầu ra là phản hồi mong đợi từ mô hình.

**TRL sẽ định dạng các tin nhắn đầu vào dựa trên định dạng chat của mô hình** Chúng cần được biểu diễn dưới dạng danh sách các từ điển với các khóa: `role` và `content`.

**Ví dụ:**
```sh
[
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing well, thank you! How can I assist you today?",},
]
```

In [20]:
# Tải dữ liệu mẫu
from datasets import load_dataset, Dataset, DatasetDict

ds = load_dataset("GAIR/o1-journey")
# Split the dataset
split_dataset = ds['train'].train_test_split(test_size=27, seed=42)

# Create a new DatasetDict with 'train' and 'test' splits
ds = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

README.md:   0%|          | 0.00/746 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/615k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/327 [00:00<?, ? examples/s]

In [21]:
# TODO: 🦁 Nếu dataset của bạn không ở định dạng mà TRL có thể chuyển đổi thành định dạng chat, bạn sẽ cần xử lý nó.
# Tham khảo [Định dạng Chat](../chat_templates.md)

def process_dataset(sample):
    # TODO: 🐢 Chuyển đổi dữ liệu sang định dạng ChatML
    # Sử dụng `tokenizer.apply_chat_template` để áp dụng định dạng hội thoại
    # 1. Tạo định dạng tin nhắn với vai trò và nội dung
    format_message = [
        {"role": "user", "content": str(sample.get('question', ''))},
        {"role": "assistant", "content": str(sample.get('longCOT', ''))},
    ]
    # 2. Áp dụng định dạng chat cho các mẫu bằng tokenizer
    sample = tokenizer.apply_chat_template(format_message, tokenize=False, add_generation_prompt=True)
    return {'text': sample}


ds = ds.map(process_dataset)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

## Điều chỉnh SFTTrainer

Điều chỉnh `SFTTrainer` với các tham số khác nhau giúp điều khiển quá trình huấn luyện trở nên hiệu quả hơn. Các thông số bao gồm
- Số bước huấn luyện (steps)
- Kích thước batch (batch size)
- Tốc độ học (learning rate)
- Chiến lược đánh giá mô hình (evaluation strategy)

Ngoài ra, còn rất nhiều thông số khác, bạn có thể tham khảo thêm ở [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)

Điều chỉnh các tham số này dựa trên yêu cầu cụ thể và tài nguyên tính toán của bạn.

In [25]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_o1_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=3e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=500,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
    max_seq_length=2048,  # Set the maximum sequence length for your model
)


# TODO: 🦁 🐕 căn chỉnh các tham số SFTTrainer với bộ dữ liệu bạn đã chọn.
# Ví dụ, nếu bạn đang sử dụng bộ `bigcode/the-stack-smol`, bạn sẽ cần chọn cột `content`

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Huấn luyện Mô hình

Với trainer đã được điều chỉnh, chúng ta có thể tiến hành huấn luyện mô hình. Quá trình huấn luyện sẽ bao gồm
- Lặp qua bộ dữ liệu
- Tính toán loss
- Cập nhật các tham số của mô hình để giảm thiểu loss này.

In [26]:
# Huấn luyện mô hình
trainer.train()

# Lưu mô hình
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,1.033000,1.019512
100,0.910000,0.983441
150,0.891300,0.967642
200,0.774900,0.966830
250,0.766100,0.979614
300,0.779300,0.976568
350,0.689800,1.003857
400,0.634000,1.032644
450,0.598200,1.034535
500,0.562500,1.075169


In [27]:
# Đưa mô hình lên Hugging Face Hub
trainer.push_to_hub(tags=finetune_tags)

events.out.tfevents.1734330678.876bc4d93c2a.1087.1:   0%|          | 0.00/5.54k [00:00<?, ?B/s]

events.out.tfevents.1734330708.876bc4d93c2a.1087.2:   0%|          | 0.00/32.4k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/quyettv/SmolLM2-FT-o1/commit/d6f7322cdce327ca9e812ee01caaced22fc06182', commit_message='End of training', commit_description='', oid='d6f7322cdce327ca9e812ee01caaced22fc06182', pr_url=None, repo_url=RepoUrl('https://huggingface.co/quyettv/SmolLM2-FT-o1', endpoint='https://huggingface.co', repo_type='model', repo_id='quyettv/SmolLM2-FT-o1'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập thêm: Sinh văn bản với mô hình vừa được huấn luyện</h2>
    <p>🐕 Sử dụng mô hình đã được huấn luyện để sinh ra phản hồi, giống như với ví dụ ban đầu.</p>
</div>

In [31]:
# Kiểm tra mô hình đã được huấn luyện trên cùng một yêu cầu

# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "George has a quadratic of the form $x^2+bx+\\frac13$, where $b$ is a specific negative number. Using his knowledge of how to complete the square, George is able to rewrite this quadratic in the form $(x+m)^2+\\frac{1}{12}$. What is $b$?"

# Định dạng chat
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Sinh phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: sử dụng mô hình đã được huấn luyện để sinh phản hồi, giống như với ví dụ.
outputs = model.generate(**inputs, max_new_tokens=2048)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
George has a quadratic of the form $x^2+bx+\frac13$, where $b$ is a specific negative number. Using his knowledge of how to complete the square, George is able to rewrite this quadratic in the form $(x+m)^2+\frac{1}{12}$. What is $b$?
assistant
Let's work through the problem step by step, exploring each part of the process carefully. We need to find the value of \(b\) for the George-Schwarz quadratic of the form \(x^2 + bx + \frac{1}{3}\).

First, we'll complete the square for this quadratic. To do this, we'll use the formula for the product of the roots of a quadratic of the form \(ax^2 + bx + \frac{1}{n}\), where \(a\) and \(b\) are specific negative numbers.

Let's start by substituting \(a = -b\) into this formula:
\[
(x^2 + bx + \frac{1}{3}) = (-b + \frac{1}{3})x^2 + bx + \frac{1}{12} = x^2 + bx + \frac{1}{3}
\]

Now, let's expand this equation:
\[
x^2 + bx + \frac{1}{3} = (x + m)^2 + \frac{1}{12}
\]

Let's pause and consider what we have so far. We've rewritt

## 💐 Chúc mừng bạn. Bạn đã hoàn thành!

Bài tập này đã cung cấp hướng dẫn từng bước để bạn huấn luyện được mô hình `HuggingFaceTB/SmolLM2-135M` sử dụng `SFTTrainer`. Bằng cách làm theo các bước này, bạn có thể điều chỉnh mô hình để thực hiện các tác vụ cụ thể hiệu quả hơn. Nếu bạn muốn tiếp tục làm việc với khóa học này, đây là một số bước bạn có thể thử:

- Thử notebook này ở mức độ khó hơn
- Review PR của học viên khác
- Cải thiện tài liệu khóa học thông qua Issue hoặc PR.